# Bayesian Grid Congestion Analysis

This notebook extends our grid congestion analysis using Bayesian statistical methods. While Monte Carlo simulation helps us understand the probability of congestion under different scenarios, Bayesian modeling allows us to:

1. Quantify uncertainty in our estimates
2. Infer the relative impact of different technologies (EVs vs. PVs)
3. Make probabilistic predictions for new scenarios
4. Incorporate prior knowledge into our analysis

The Bayesian approach provides a more rigorous statistical framework for understanding the factors that contribute to grid congestion.

## Setup and Configuration

First, we'll import the necessary libraries and define our simulation parameters. We'll use PyMC for Bayesian modeling and ArviZ for visualization of Bayesian inference results.

In [9]:
# Add to imports
import pymc as pm
import arviz as az
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('..')  # Add parent directory to path

# Parameters
N_HOUSES = 50
HOURS = 24
TRANSFORMER_CAPACITY = 125  # kW
TRANSFORMER_ID = "TR_001"
DATA_DIR = "data/generated"
RESULTS_FILE = f"{DATA_DIR}/monte_carlo_results.csv"
N_SIMULATIONS = 200

## Bayesian Model Definition

Here we define a Bayesian logistic regression model to predict congestion probability based on the number of EVs and PVs. The model includes:

- An intercept term (alpha) representing the baseline congestion risk
- Coefficients for EV and PV counts (beta_ev, beta_pv) representing their impact on congestion
- A noise term (sigma) to account for unexplained variation

We standardize the predictors (EV and PV counts) to improve model convergence and make the coefficients more interpretable.

In [10]:
def run_bayesian_congestion_model(results_df, n_samples=1000):
    """
    Build a Bayesian model to predict congestion probability based on EV and PV counts
    
    Args:
        results_df: DataFrame with simulation results
        n_samples: Number of samples for MCMC
    
    Returns:
        Inference data object with posterior samples and standardization parameters
    """
    # Extract data
    n_ev = results_df['n_ev'].values
    n_pv = results_df['n_pv'].values
    congestion = results_df['congestion_probability'].values
    
    # Standardize predictors
    ev_mean, ev_std = n_ev.mean(), n_ev.std()
    pv_mean, pv_std = n_pv.mean(), n_pv.std()
    
    n_ev_std = (n_ev - ev_mean) / ev_std
    n_pv_std = (n_pv - pv_mean) / pv_std
    
    # Define model
    with pm.Model() as model:
        # Priors
        alpha = pm.Normal('alpha', mu=0, sigma=1)
        beta_ev = pm.Normal('beta_ev', mu=0, sigma=1)
        beta_pv = pm.Normal('beta_pv', mu=0, sigma=1)
        sigma = pm.HalfNormal('sigma', sigma=1)
        
        # Linear predictor (logit scale)
        mu = alpha + beta_ev * n_ev_std + beta_pv * n_pv_std
        
        # Likelihood (Beta distribution for probabilities)
        theta = pm.Deterministic('theta', pm.math.invlogit(mu))
        y = pm.Beta('y', alpha=theta*sigma, beta=(1-theta)*sigma, observed=congestion)
        
        # Sample from posterior
        idata = pm.sample(n_samples, return_inferencedata=True)
    
    return idata, ev_mean, ev_std, pv_mean, pv_std

## Visualization Function

This function creates a contour plot showing the predicted congestion probability across different combinations of EV and PV adoption. It uses the posterior distribution from our Bayesian model to make these predictions.

In [11]:
def plot_bayesian_contour(idata, results_df, ev_mean, ev_std, pv_mean, pv_std, output_dir=DATA_DIR):
    """Plot contour map of congestion probability from Bayesian model"""
    # Create prediction grid for smooth surface
    ev_range = np.linspace(0, results_df['n_ev'].max(), 100)
    pv_range = np.linspace(0, results_df['n_pv'].max(), 100)
    ev_grid, pv_grid = np.meshgrid(ev_range, pv_range)
    
    # Standardize grid values
    ev_grid_std = (ev_grid - ev_mean) / ev_std
    pv_grid_std = (pv_grid - pv_mean) / pv_std
    
    # Get posterior samples
    posterior = idata.posterior
    alpha = posterior['alpha'].values.flatten()
    beta_ev = posterior['beta_ev'].values.flatten()
    beta_pv = posterior['beta_pv'].values.flatten()
    
    # Compute predicted probabilities (mean of posterior)
    logit_p = (alpha.mean() + 
              beta_ev.mean() * ev_grid_std + 
              beta_pv.mean() * pv_grid_std)
    
    prob = 1 / (1 + np.exp(-logit_p))
    
    # Create detailed contour plot with data points
    plt.figure(figsize=(10, 8))
    contour = plt.contourf(ev_grid, pv_grid, prob, cmap='YlOrRd', levels=20)
    plt.colorbar(label='Congestion Probability')
    
    # Add contour lines
    contour_lines = plt.contour(ev_grid, pv_grid, prob, colors='black', alpha=0.5, 
                               levels=[0.1, 0.3, 0.5, 0.7, 0.9])
    plt.clabel(contour_lines, inline=True, fontsize=8)
    
    # Overlay actual data points
    sc = plt.scatter(results_df['n_ev'], results_df['n_pv'], 
                    c=results_df['congestion_probability'], 
                    cmap='YlOrRd', edgecolor='black', s=50)
    
    plt.title('Bayesian Model: Grid Congestion Probability')
    plt.xlabel('Number of EVs')
    plt.ylabel('Number of Solar PV')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/bayesian_contour_detailed.png", dpi=300)
    plt.show()
    
    # Create simplified contour plot for presentations
    plt.figure(figsize=(8, 6))
    contour = plt.contourf(ev_grid, pv_grid, prob, cmap='YlOrRd', levels=10)
    plt.colorbar(label='Congestion Probability')
    
    # Add the 50% probability contour line
    contour_50 = plt.contour(ev_grid, pv_grid, prob, colors='blue', 
                            levels=[0.5], linewidths=2)
    plt.clabel(contour_50, inline=True, fontsize=10, fmt='%.2f')
    
    # Add a text annotation for the threshold
    mid_ev = results_df['n_ev'].max() / 2
    mid_pv = results_df['n_pv'].max() / 2
    plt.annotate('50% Risk Threshold', 
                 xy=(mid_ev, mid_pv),
                 xytext=(mid_ev + 2, mid_pv + 2),
                 arrowprops=dict(facecolor='blue', shrink=0.05),
                 fontsize=10,
                 color='blue')
    
    plt.title('Grid Congestion Risk by Technology Adoption')
    plt.xlabel('Number of EVs')
    plt.ylabel('Number of Solar PV')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/bayesian_contour_simple.png", dpi=300)
    plt.show()

## Run the analysis

Now we'll run the Bayesian analysis on our Monte Carlo simulation results. This includes:

1. Loading the simulation results
2. Running the Bayesian model
3. Visualizing the posterior distributions
4. Creating contour plots of predicted congestion probability
5. Calculating and interpreting effect sizes

In [12]:

results = pd.read_csv(RESULTS_FILE)
print(f"Loaded results with {len(results)} scenarios")

# Run Bayesian model
print("\nRunning Bayesian model...")
idata, ev_mean, ev_std, pv_mean, pv_std = run_bayesian_congestion_model(results)


Loaded results with 36 scenarios

Running Bayesian model...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'alpha': array(0.43473893), 'beta_ev': array(0.15017343), 'beta_pv': array(-0.48246548), 'sigma_log__': array(-0.71643459)}

Logp initial evaluation results:
{'alpha': -1.01, 'beta_ev': -0.93, 'beta_pv': -1.04, 'sigma': -1.06, 'y': inf}
You can call `model.debug()` for more details.

## Examine results

Let's execute the main function to run our Bayesian analysis on the Monte Carlo simulation results.

In [ ]:

# Summarize posterior
print("\nBayesian Model Summary:")
print(az.summary(idata))

# Plot posterior distributions for specific parameters
az.plot_posterior(idata, var_names=['alpha', 'beta_ev', 'beta_pv', 'sigma'])
plt.suptitle("Posterior Distributions")
plt.tight_layout()
plt.savefig(f"{DATA_DIR}/bayesian_posterior.png")
plt.show()

# Plot contour map
plot_bayesian_contour(idata, results, ev_mean, ev_std, pv_mean, pv_std)

# Calculate and print effect sizes
posterior = idata.posterior
beta_ev = posterior['beta_ev'].values.flatten()
beta_pv = posterior['beta_pv'].values.flatten()

print("\nEffect Sizes:")
print(f"EV effect (mean): {beta_ev.mean():.4f}")
print(f"EV effect (95% CI): [{np.percentile(beta_ev, 2.5):.4f}, {np.percentile(beta_ev, 97.5):.4f}]")
print(f"PV effect (mean): {beta_pv.mean():.4f}")
print(f"PV effect (95% CI): [{np.percentile(beta_pv, 2.5):.4f}, {np.percentile(beta_pv, 97.5):.4f}]")

# Calculate probability that EV effect is greater than PV effect (in absolute terms)
prob_ev_greater = np.mean(np.abs(beta_ev) > np.abs(beta_pv))
print(f"Probability |EV effect| > |PV effect|: {prob_ev_greater:.4f}")


## Interpreting the Results

The Bayesian analysis provides several key insights:

1. **Effect Sizes**: The coefficients (beta_ev, beta_pv) tell us the relative impact of each technology on congestion risk. A positive coefficient increases risk, while a negative coefficient decreases it.

2. **Uncertainty Quantification**: The 95% credible intervals show our uncertainty about these effects. Narrower intervals indicate more confident estimates.

3. **Comparative Impact**: The probability that |EV effect| > |PV effect| tells us how confident we are that EVs have a stronger impact (positive or negative) than PVs on congestion risk.

4. **Predictive Surface**: The contour plots show the predicted congestion probability across different combinations of EV and PV adoption, with the 50% threshold highlighted as a decision boundary.

This analysis helps grid planners understand not just whether congestion might occur, but the relative contribution of different technologies and the uncertainty in these estimates.